# 신경망 모델 구성하기 

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
#학습을 위한 장치 열기
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
#클래스 정의하기
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
#모델의 구조 출력
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 입력 데이터를 전달합니다. 이는 일부 백그라운드 연산들 과 함께 모델의 forward 를 실행합니다. model.forward() 를 직접 호출하지 마세요!

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10-차원 텐서가 반환됩니다. 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([3], device='cuda:0')


### 모델 계층(layer)

In [7]:
#28*28크기의 이미지 3개로 구성된 미니배치 신경망 통과 시뮬레이션
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
#28*28의 2D이미지를 784 픽셀 값을 갖는 연속된 배열로 변환  
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
#선형 계층 
#입력에 선형 변환(weight, bisa)을 적용 
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
#activation은 모델의 입력과 출력 사이에 복잡한 관계를 만듬. (nonlinearity) 다양한 현상을 학습할 수 있도록 만듬
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.5866,  0.6947, -0.8417,  0.1154, -0.4559,  0.2367,  1.0495, -0.0749,
         -0.3703,  0.7652,  0.2360,  0.3611,  0.4860,  0.3277, -0.2386,  0.6463,
          0.3842, -0.1584,  0.2828, -0.0638],
        [ 0.7600,  0.2559, -0.5853,  0.1515, -0.1535,  0.1953,  0.8595, -0.1100,
         -0.6214,  0.7946,  0.2514,  0.1712,  0.5331,  0.1717, -0.1561,  0.5673,
          0.0325, -0.1686,  0.0041, -0.0610],
        [ 0.8618,  0.2407, -0.4922,  0.3402, -0.2686, -0.1487,  0.7711,  0.0038,
         -0.6711,  0.2950,  0.5205,  0.3314,  0.7845,  0.2958, -0.5857,  0.6550,
          0.2166, -0.3077,  0.2384,  0.0824]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.5866, 0.6947, 0.0000, 0.1154, 0.0000, 0.2367, 1.0495, 0.0000, 0.0000,
         0.7652, 0.2360, 0.3611, 0.4860, 0.3277, 0.0000, 0.6463, 0.3842, 0.0000,
         0.2828, 0.0000],
        [0.7600, 0.2559, 0.0000, 0.1515, 0.0000, 0.1953, 0.8595, 0.0000, 0.0000,
         0.7946, 0.2514, 0.1712, 0.5331, 0.1717, 0.000

In [11]:
#nn.Sequential 은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 
#순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [12]:
#신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다.
#logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됩니다. 
#dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 파라미터

신경망 내부의 많은 계층들은 매개변수화(parameterize) 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

In [13]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0294, -0.0248,  0.0237,  ..., -0.0055, -0.0057, -0.0188],
        [ 0.0098,  0.0184, -0.0162,  ...,  0.0212,  0.0218, -0.0208]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0218, 0.0098], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0037, -0.0420, -0.0028,  ...,  0.0321,  0.0212,  0.0188],
        [-0.0264,  0.0414, -0.0066,  ...,  0.0116,  0.0343, -0.0406]],
       device='cuda:0